In [10]:
import numpy as np 
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
import cv2
import csv
import sys
import os
#from PIL import Image
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor
from tqdm.notebook import tqdm
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Function

#from tqdm.autonotebook import tqdm
#from tqdm import tqdm


In [7]:
class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()

        # define: encoder
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3, 16, 3, 2),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(2),
            nn.LeakyReLU(0.01, inplace=True),
            nn.Conv2d(16, 32, 3, 2),
            nn.BatchNorm2d(32),
            nn.LeakyReLU(0.01, inplace=True),
            nn.Conv2d(32, 64, 3, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.01, inplace=True)
        )
 
        self.classifier = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(32, 10),
        )


    
    def forward(self, x):
        features = self.feature_extractor(x)
        features = features.view(x.shape[0], -1)
        logits = self.classifier(features)
        return logits

In [41]:
# detect is gpu available.
use_gpu = torch.cuda.is_available()
if use_gpu:
    device =torch.device('cuda:0')
else:
    device =torch.device("cpu")
class Image_data(Dataset):
    def __init__(self,label,data):
        self.data_path = data
        self.label_path = label
    def __getitem__(self,index):

        trainX = np.load(self.data_path)
        trainY = np.load(self.label_path)
        trainX = np.transpose(trainX, (0, 3, 1, 2))
        trainX = torch.Tensor(trainX)
        trainY = torch.Tensor(trainY)
        return trainX[index],trainY[index]

    def __len__(self):
        return len(np.load(self.label_path))

train_data = Image_data('./trainY.npy','./trainX.npy')



In [33]:
batch_size = 32
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

#train_dataloader = DataLoader(train_data, batch_size=batch_size,sampler=train_sampler)
#val_dataloader = DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler)


train_dataloader = DataLoader(train_data,batch_size = 32)

In [34]:
test_data = np.load('testX.npy')


rgb_test_data = []
for image in test_data:
    padding =np.zeros((image.shape[0]+4,image.shape[1]+4),dtype = np.uint8)
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            padding[i+1][j+1] = image[i][j]
    color_img = cv2.cvtColor(padding,cv2.COLOR_GRAY2RGB)
    rgb_test_data.append(color_img)
rgb_test_data = np.array(rgb_test_data)
test_data = np.transpose(rgb_test_data, (0, 3, 1, 2))
test_data = torch.Tensor(test_data)

print(test_data.shape)

torch.Size([100000, 3, 32, 32])


In [35]:
class GradientReversalFunction(Function):
    """
    Gradient Reversal Layer from:
    Unsupervised Domain Adaptation by Backpropagation (Ganin & Lempitsky, 2015)
    Forward pass is the identity function. In the backward pass,
    the upstream gradients are multiplied by -lambda (i.e. gradient is reversed)
    """

    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x.clone()

    @staticmethod
    def backward(ctx, grads):
        lambda_ = ctx.lambda_
        lambda_ = grads.new_tensor(lambda_)
        dx = -lambda_ * grads
        return dx, None


class GradientReversal(torch.nn.Module):
    def __init__(self, lambda_=1):
        super(GradientReversal, self).__init__()
        self.lambda_ = lambda_

    def forward(self, x):
        return GradientReversalFunction.apply(x, self.lambda_)

In [44]:
clf_model = Net()
clf_model.load_state_dict(torch.load("./model.pt"))
import argparse

import torch
from torch import nn
from torch.autograd import grad
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor
from tqdm.notebook import  trange

#import config
#from data import MNISTM
#from models import Net
#from utils import loop_iterable, set_requires_grad, GrayscaleToRgb


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def set_requires_grad(model, requires_grad=True):
    for param in model.parameters():
        param.requires_grad = requires_grad

def loop_iterable(iterable):
    while True:
        yield from iterable

def gradient_penalty(critic, h_s, h_t):
    # based on: https://github.com/caogang/wgan-gp/blob/master/gan_cifar10.py#L116
    alpha = torch.rand(h_s.size(0), 1).to(device)
    differences = h_t - h_s
    interpolates = h_s + (alpha * differences)
    interpolates = torch.stack([interpolates, h_s, h_t]).requires_grad_()

    preds = critic(interpolates)
    gradients = grad(preds, interpolates,
                     grad_outputs=torch.ones_like(preds),
                     retain_graph=True, create_graph=True)[0]
    gradient_norm = gradients.norm(2, dim=1)
    gradient_penalty = ((gradient_norm - 1)**2).mean()
    return gradient_penalty


feature_extractor = clf_model.feature_extractor
discriminator = clf_model.classifier

critic = nn.Sequential(
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, 10),
    nn.ReLU(),
    nn.Linear(10, 1)
).to(device)

half_batch = 32
train_data = Image_data('./trainY.npy','./trainX.npy')
source_dataset = train_data
source_loader = DataLoader(source_dataset, batch_size=half_batch, drop_last=True,shuffle=True, num_workers=0, pin_memory=True)

target_dataset = test_data 
target_loader = DataLoader(target_dataset, batch_size=half_batch, drop_last=True,shuffle=True, num_workers=0, pin_memory=True)

critic_optim = torch.optim.Adam(critic.parameters(), lr=1e-4)
clf_optim = torch.optim.Adam(clf_model.parameters(), lr=1e-4)
clf_criterion = nn.CrossEntropyLoss()

for epoch in range(1, 5+1):
    batch_iterator = zip(loop_iterable(source_loader), loop_iterable(target_loader))

    total_loss = 0
    total_accuracy = 0
    for _ in trange(500, leave=False):
        (source_x, source_y), (target_x) = next(batch_iterator)
        # Train critic
        set_requires_grad(feature_extractor, requires_grad=False)
        set_requires_grad(critic, requires_grad=True)

        source_x, target_x = source_x.to(device), target_x.to(device)
        source_y = source_y.to(device)
        source_y = source_y.long()
        with torch.no_grad():
            h_s = feature_extractor(source_x).data.view(source_x.shape[0], -1)
            h_t = feature_extractor(target_x).data.view(target_x.shape[0], -1)
        for _ in range(5):
            gp = gradient_penalty(critic, h_s, h_t)

            critic_s = critic(h_s)
            critic_t = critic(h_t)
            wasserstein_distance = critic_s.mean() - critic_t.mean()

            critic_cost = -wasserstein_distance + 10*gp

            critic_optim.zero_grad()
            critic_cost.backward()
            critic_optim.step()

            total_loss += critic_cost.item()

        # Train classifier
        set_requires_grad(feature_extractor, requires_grad=True)
        set_requires_grad(critic, requires_grad=False)
        for _ in range(1):
            source_features = feature_extractor(source_x).view(source_x.shape[0], -1)
            target_features = feature_extractor(target_x).view(target_x.shape[0], -1)

            source_preds = discriminator(source_features)
            clf_loss = clf_criterion(source_preds, source_y)
            wasserstein_distance = critic(source_features).mean() - critic(target_features).mean()

            loss = clf_loss + 1 * wasserstein_distance
            clf_optim.zero_grad()
            loss.backward()
            clf_optim.step()

    mean_loss = total_loss / (500 * 5)
    tqdm.write(f'EPOCH {epoch:03d}: critic_loss={mean_loss:.4f}')
    torch.save(clf_model.state_dict(), 'trained_models/wdgrl.pt')


 73%|███████▎  | 363/500 [20:29<03:30,  1.53s/it]

EPOCH 001: critic_loss=1.3480


 73%|███████▎  | 363/500 [32:14<03:30,  1.53s/it]

EPOCH 002: critic_loss=-0.3683


 73%|███████▎  | 363/500 [44:26<03:30,  1.53s/it]

EPOCH 003: critic_loss=-0.3036


 73%|███████▎  | 363/500 [56:35<03:30,  1.53s/it]

EPOCH 004: critic_loss=-0.3127


 73%|███████▎  | 363/500 [1:08:15<03:30,  1.53s/it]

EPOCH 005: critic_loss=-0.2557


In [45]:
test_data = torch.Tensor(test_data)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
clf_model.eval()
ans = []
for idx ,img in enumerate(test_loader):
    ans.append(clf_model(img))
ans_final =[]
for a in ans:
    for pre in a:
        pre = pre.tolist()
        ans_final.append(pre.index(max(pre)))
    

In [46]:
with open("wdgrl.csv","w") as f:
    print("id,label", file = f)
    for id,label in enumerate(ans_final):
        print("{},{}".format(id,label) ,file = f)